# 訓練集的訓練（Cresci-17）
## 第一部，語言檢測
先進行語言檢測，看看是否存在中文的元素
首先載入需要的模型

In [35]:
import fasttext
import json
import re
import opencc
import jieba
import jieba.analyse
import pandas as pd
import numpy as np

from math import log, floor
from lingua import Language, LanguageDetectorBuilder
from matplotlib import pyplot
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.metrics import classification_report

languages = [Language.ENGLISH, Language.FRENCH, Language.GERMAN, Language.SPANISH, Language.JAPANESE, Language.CHINESE, Language.ARABIC]
detector = LanguageDetectorBuilder.from_languages(*languages).with_preloaded_language_models().build()
converter = opencc.OpenCC('s2tw.json')
pd.options.display.max_rows = 200
pd.set_option('display.max_columns', None)  
# ft_model = fasttext.load_model("./fasttext/lid.176.bin")
jieba.set_dictionary('./jieba/dict.txt')

In [10]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


讀入真人帳戶，進行語言判斷

In [13]:
genuine_account_list = []
text_obj = []
with open("./cresci-17/genuine_accounts.csv/tweets.csv", "rb") as genuine_csv:
    genuine_sheets = pd.read_csv(genuine_csv, encoding="UTF-8", encoding_errors='replace')
    genuine_csv.close()

C:\UserData\AppData\Local\Temp\ipykernel_3836\1329798845.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  genuine_sheets = pd.read_csv(genuine_csv, encoding="UTF-8", encoding_errors='replace')


In [14]:
with open('./cresci-17/social_spambots_1.csv/tweets.csv', "rb") as spambots1_csv:
    spambots1_sheets = pd.read_csv(spambots1_csv, encoding="UTF-8", encoding_errors='replace')
    spambots1_csv.close()
with open('./cresci-17/social_spambots_2.csv/tweets.csv', "rb") as spambots2_csv:
    spambots2_sheets = pd.read_csv(spambots2_csv, encoding="UTF-8", encoding_errors='replace')
    spambots2_csv.close()
with open('./cresci-17/social_spambots_3.csv/tweets.csv', "rb") as spambots3_csv:
    spambots3_sheets = pd.read_csv(spambots3_csv, encoding="UTF-8", encoding_errors='replace')
    spambots3_csv.close()
spambots_sheets = pd.concat([spambots1_sheets, spambots2_sheets, spambots3_sheets])

C:\UserData\AppData\Local\Temp\ipykernel_3836\3688565683.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  spambots1_sheets = pd.read_csv(spambots1_csv, encoding="UTF-8", encoding_errors='replace')
C:\UserData\AppData\Local\Temp\ipykernel_3836\3688565683.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  spambots2_sheets = pd.read_csv(spambots2_csv, encoding="UTF-8", encoding_errors='replace')
C:\UserData\AppData\Local\Temp\ipykernel_3836\3688565683.py:8: DtypeWarning: Columns (7,10) have mixed types. Specify dtype option on import or set low_memory=False.
  spambots3_sheets = pd.read_csv(spambots3_csv, encoding="UTF-8", encoding_errors='replace')


In [17]:
genuine_sheets.shape

(2839362, 25)

In [16]:
spambots_sheets.shape

(3457133, 25)

In [18]:
genuine_sheets.sample(n=100).reset_index().drop('index', axis=1)

,id,text,source,user_id,truncated,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,retweeted_status_id,geo,place,contributors,retweet_count,reply_count,favorite_count,favorited,retweeted,possibly_sensitive,num_hashtags,num_urls,num_mentions,created_at,timestamp,crawled_at,updated
0,507923761215991808,The canal in the sky! Love &amp; amazed by the...,"<a href=""http://instagram.com"" rel=""nofollow"">...",1.485158e+07,NaN,0.000000e+00,0.000000e+00,NaN,0.000000e+00,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,1.0,0.0,Fri Sep 05 16:10:36 +0000 2014,2014-09-05 18:10:36,2015-05-01 13:22:11,2015-05-01 13:22:11
1,592595668562153472,RT @BinodKChaudhary: Wai Wai being dispatched ...,"<a href=""http://twitter.com/download/android"" ...",2.252819e+08,NaN,0.000000e+00,0.000000e+00,NaN,5.925626e+17,NaN,NaN,NaN,71.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,1.0,Mon Apr 27 07:46:31 +0000 2015,2015-04-27 09:46:31,2015-05-01 15:57:14,2015-05-01 15:57:14
2,476256119786975232,Can you guess what this is? #idontknow #icantd...,"<a href=""http://instagram.com"" rel=""nofollow"">...",1.646845e+07,NaN,0.000000e+00,0.000000e+00,NaN,0.000000e+00,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,2.0,1.0,0.0,Tue Jun 10 06:54:42 +0000 2014,2014-06-10 08:54:42,2015-05-01 13:35:06,2015-05-01 13:35:06
3,474930561983598593,Aaaa teens pala ha? Ee mga regular housemates ...,"<a href=""http://twitter.com/download/android"" ...",2.326385e+09,NaN,0.000000e+00,0.000000e+00,NaN,0.000000e+00,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,Fri Jun 06 15:07:24 +0000 2014,2014-06-06 17:07:24,2015-05-01 16:06:37,2015-05-01 16:06:37
4,582662843004104704,@iReplayPortugal gold star for meh?,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",2.223970e+09,NaN,5.826627e+17,2.715384e+09,iReplayPortugal,0.000000e+00,NaN,NaN,NaN,0.0,0.0,1.0,NaN,NaN,NaN,0.0,0.0,1.0,Mon Mar 30 21:57:01 +0000 2015,2015-03-30 23:57:01,2015-05-01 14:15:43,2015-05-01 14:15:43
5,592314517628985346,I know its kinda weird that they kissed but i ...,"<a href=""http://twitter.com/download/android"" ...",2.401671e+08,NaN,0.000000e+00,0.000000e+00,NaN,0.000000e+00,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,2.0,0.0,2.0,Sun Apr 26 13:09:20 +0000 2015,2015-04-26 15:09:20,2015-05-01 16:12:31,2015-05-01 16:12:31
6,593435643658772480,I need you girl wae honja saranghago honjaseom...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",2.323691e+09,NaN,0.000000e+00,0.000000e+00,NaN,0.000000e+00,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,Wed Apr 29 15:24:17 +0000 2015,2015-04-29 17:24:17,2015-05-01 16:04:50,2015-05-01 16:04:50
7,449751668704292864,@J_Jammer @jimmykdoyle cats playing country mu...,"<a href=""http://twitter.com/download/iphone"" r...",1.726110e+07,NaN,4.497502e+17,3.130793e+07,J_Jammer,0.000000e+00,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,2.0,Sat Mar 29 03:35:28 +0000 2014,2014-03-29 04:35:28,2015-05-01 13:40:43,2015-05-01 13:40:43
8,451481918597378048,RT @RyanMauro: ALERT: SHOOTING AT FORT HOOD ht...,"<a href=""http://www.metrotwit.com/"" rel=""nofol...",1.509788e+07,NaN,0.000000e+00,0.000000e+00,NaN,4.514813e+17,NaN,NaN,NaN,6.0,0.0,0.0,NaN,NaN,NaN,0.0,1.0,1.0,Wed Apr 02 22:10:52 +0000 2014,2014-04-03 00:10:52,2015-05-01 13:26:17,2015-05-01 13:26:17
9,582365685222903808,@Michael5SOS I LOVE YOU,"<a href=""http://twitter.com/download/iphone"" r...",2.263715e+09,NaN,5.823654e+17,4.032468e+08,Michael5SOS,0.000000e+00,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,1.0,Mon Mar 30 02:16:13 +0000 2015,2015-03-30 04:16:13,2015-05-01 15:05:37,2015-05-01 15:05:37


對貼文內容進行除雜，參照[spam_identifier](https://github.com/mansoor9743/spam_identifier)的方法，去除rt資訊，並將標點改為空白，提取出乾淨的資訊

In [19]:
def msg_clean(original: str, dirty: bool = False) -> str:
    # text_prediction = detector.detect_language_of(output)
    # if (not dirty) and text_prediction[0][0] == "__label__zh" and text_prediction[1][0] >= 0.7:
    #     # If the original language is Chinese, force convert to traditional Chinese
    #     original = converter.convert(original)
    # Patterns
    url_pattern = r'https?:\/\/(?:www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b(?:[-a-zA-Z0-9()@:%_\+.~#?&\/=]*)'
    rt_at_pattern = r'((?:RT )?@[0-9A-Za-z]+)'
    punctuation_pattern = r'[~!@#$%^&*\(\)_+=\-`\[\]{}\';:\".,<>/?\|]'
    punctuation_zh_pattern = r'[\u3000\u3001-\u303F]'
    punctuation_fullshape_pattern= r'[\uFF01-\uFF5E]'
    hashtag_pattern = r'(#\w*)'
    # Pattern removal
    output = re.sub(rt_at_pattern, "", original)
    output = re.sub(url_pattern, "", output)
    output = re.sub(hashtag_pattern, "", output)
    if not dirty:
        output = re.sub(punctuation_pattern, "", output)
        output = re.sub(punctuation_zh_pattern, " ", output)
        output = re.sub(punctuation_fullshape_pattern, " ", output)
    return output

In [20]:
def get_entropy(original: str) -> float:
    word_list = [x for x in jieba.cut(original, cut_all=False) if not x == ' ']
    word_freq = {}
    for word in word_list:
        if word_freq.get(word) is None: word_freq[word] = 1
        else: word_freq[word] += 1
    for item in word_freq.keys():
        word_freq[item] = word_freq[item] / len(word_freq)
    entropy = 0
    for value in word_freq.values():
        entropy -= value * log(value)
    return entropy

In [36]:
def parse_text(original: str, retweet_count: int = 0, reply_count: int = 0, favorite_count: int = 0):
    # language detection
    language = detector.detect_language_of(original)
    language_name = str(language.name) if language is not None else "None"
    # Priority: At, Hashtag, URL links.
    url_pattern = r'(https?:\/\/(?:www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b(?:[-a-zA-Z0-9()@:%_\+.~#?&\/=]*))'
    rt_at_pattern = r'((?:RT )?@[0-9A-Za-z_]+)'
    hashtag_pattern = r' ((#|＃)([a-z0-9_\u00c0-\u00d6\u00d8-\u00f6\u00f8-\u00ff\u0100-\u024f\u0253-\u0254\u0256-\u0257\u0300-\u036f\u1e00-\u1eff\u0400-\u04ff\u0500-\u0527\u2de0-\u2dff\ua640-\ua69f\u0591-\u05bf\u05c1-\u05c2\u05c4-\u05c5\u05d0-\u05ea\u05f0-\u05f4\ufb12-\ufb28\ufb2a-\ufb36\ufb38-\ufb3c\ufb40-\ufb41\ufb43-\ufb44\ufb46-\ufb4f\u0610-\u061a\u0620-\u065f\u066e-\u06d3\u06d5-\u06dc\u06de-\u06e8\u06ea-\u06ef\u06fa-\u06fc\u0750-\u077f\u08a2-\u08ac\u08e4-\u08fe\ufb50-\ufbb1\ufbd3-\ufd3d\ufd50-\ufd8f\ufd92-\ufdc7\ufdf0-\ufdfb\ufe70-\ufe74\ufe76-\ufefc\u200c-\u200c\u0e01-\u0e3a\u0e40-\u0e4e\u1100-\u11ff\u3130-\u3185\ua960-\ua97f\uac00-\ud7af\ud7b0-\ud7ff\uffa1-\uffdc\u30a1-\u30fa\u30fc-\u30fe\uff66-\uff9f\uff10-\uff19\uff21-\uff3a\uff41-\uff5a\u3041-\u3096\u3099-\u309e\u3400-\u4dbf\u4e00-\u9fff\u20000-\u2a6df\u2a700-\u2b73f\u2b740-\u2b81f\u2f800-\u2fa1f]*[a-z_\u00c0-\u00d6\u00d8-\u00f6\u00f8-\u00ff\u0100-\u024f\u0253-\u0254\u0256-\u0257\u0300-\u036f\u1e00-\u1eff\u0400-\u04ff\u0500-\u0527\u2de0-\u2dff\ua640-\ua69f\u0591-\u05bf\u05c1-\u05c2\u05c4-\u05c5\u05d0-\u05ea\u05f0-\u05f4\ufb12-\ufb28\ufb2a-\ufb36\ufb38-\ufb3c\ufb40-\ufb41\ufb43-\ufb44\ufb46-\ufb4f\u0610-\u061a\u0620-\u065f\u066e-\u06d3\u06d5-\u06dc\u06de-\u06e8\u06ea-\u06ef\u06fa-\u06fc\u0750-\u077f\u08a2-\u08ac\u08e4-\u08fe\ufb50-\ufbb1\ufbd3-\ufd3d\ufd50-\ufd8f\ufd92-\ufdc7\ufdf0-\ufdfb\ufe70-\ufe74\ufe76-\ufefc\u200c-\u200c\u0e01-\u0e3a\u0e40-\u0e4e\u1100-\u11ff\u3130-\u3185\ua960-\ua97f\uac00-\ud7af\ud7b0-\ud7ff\uffa1-\uffdc\u30a1-\u30fa\u30fc-\u30fe\uff66-\uff9f\uff10-\uff19\uff21-\uff3a\uff41-\uff5a\u3041-\u3096\u3099-\u309e\u3400-\u4dbf\u4e00-\u9fff\u20000-\u2a6df\u2a700-\u2b73f\u2b740-\u2b81f\u2f800-\u2fa1f][a-z0-9_\u00c0-\u00d6\u00d8-\u00f6\u00f8-\u00ff\u0100-\u024f\u0253-\u0254\u0256-\u0257\u0300-\u036f\u1e00-\u1eff\u0400-\u04ff\u0500-\u0527\u2de0-\u2dff\ua640-\ua69f\u0591-\u05bf\u05c1-\u05c2\u05c4-\u05c5\u05d0-\u05ea\u05f0-\u05f4\ufb12-\ufb28\ufb2a-\ufb36\ufb38-\ufb3c\ufb40-\ufb41\ufb43-\ufb44\ufb46-\ufb4f\u0610-\u061a\u0620-\u065f\u066e-\u06d3\u06d5-\u06dc\u06de-\u06e8\u06ea-\u06ef\u06fa-\u06fc\u0750-\u077f\u08a2-\u08ac\u08e4-\u08fe\ufb50-\ufbb1\ufbd3-\ufd3d\ufd50-\ufd8f\ufd92-\ufdc7\ufdf0-\ufdfb\ufe70-\ufe74\ufe76-\ufefc\u200c-\u200c\u0e01-\u0e3a\u0e40-\u0e4e\u1100-\u11ff\u3130-\u3185\ua960-\ua97f\uac00-\ud7af\ud7b0-\ud7ff\uffa1-\uffdc\u30a1-\u30fa\u30fc-\u30fe\uff66-\uff9f\uff10-\uff19\uff21-\uff3a\uff41-\uff5a\u3041-\u3096\u3099-\u309e\u3400-\u4dbf\u4e00-\u9fff]*))'
    word_list = [x for x in jieba.cut(msg_clean(original, dirty=True), cut_all=False) if not x == ' ']
    print(word_list)
    word_count = len(word_list)
    url_count = re.findall(url_pattern, original)
    hashtag_count = re.findall(hashtag_pattern, original)
    rt_at_count = re.findall(rt_at_pattern, original)
    
    
    clean_text = msg_clean(original)
    wordlist_text = " ".join(word_list)
    entropy = get_entropy(clean_text)
    language_index = {'ENGLISH': 0, 'FRENCH': 1, 'GERMAN': 2, 'SPANISH': 3, 'JAPANESE': 4, 'CHINESE': 5, 'ARABIC': 6, 'None': 7}
    language_detect = detector.detect_language_of(msg_clean(original, dirty=True))
    languages = language_index[str(language_detect.name) if language_detect is not None else 'None']
    
    packed_obj = {
        "original_text": original,
        "cleaned_text": clean_text,
        "wordlist_text": wordlist_text,
        "word_count": word_count,
        "num_urls": len(url_count) if url_count is not None else 0,
        "num_hashtags": len(hashtag_count) if hashtag_count is not None else 0,
        "num_mentions": len(rt_at_count) if rt_at_count is not None else 0,
        "num_retweets": retweet_count,
        "num_replys": reply_count,
        "num_favorite": favorite_count,
        "lang_code": languages,
        "entropy": entropy
    }
    print(rt_at_count)
    return packed_obj
    

In [31]:
    print(get_entropy(msg_clean("ust played his first game of rugby happy and")))

2.1972245773362196


In [65]:
# Remove non-English contents
genuine_tweets = pd.DataFrame()
count = 0
percentage = 0
language_index = {'ENGLISH': 18, 'FRENCH': 22, 'GERMAN': 25, 'SPANISH': 59, 'JAPANESE': 35, 'CHINESE': 5, 'ARABIC': 3, 'None': 7}
for rows in genuine_sheets.sample(n=20000).itertuples():
    count += 1
    last_percentage = percentage
    percentage = floor(count / 20000 * 1000) / 10
    row_obj = pd.Series(rows._asdict())
    if not (percentage == last_percentage): print("{:.1f}%..".format(percentage), end="")
    text_prediction = detector.detect_language_of(msg_clean(str(row_obj['text']), dirty=True))
    # print(language_index[text_prediction.name])
    row_obj['languages'] = str(text_prediction.name) if text_prediction is not None else 'None'
    row_obj['confidence'] = str(language_index[text_prediction.name]) if text_prediction is not None else 'None'
#    row_obj['confidence'] = str(text_prediction.value) if text_prediction is not None else 'None'
    if len(genuine_tweets) == 0: 
        genuine_tweets = pd.DataFrame([row_obj])
        continue
    genuine_tweets.loc[len(genuine_tweets)] = row_obj
genuine_tweets = genuine_tweets.reset_index().drop('Index', axis=1)
genuine_tweets

0.1%..0.2%..0.3%..0.4%..0.5%..0.6%..0.7%..0.8%..0.9%..1.0%..1.1%..1.2%..1.3%..1.4%..1.5%..1.6%..1.7%..1.8%..1.9%..2.0%..2.1%..2.2%..2.3%..2.4%..2.5%..2.6%..2.7%..2.8%..2.9%..3.0%..3.1%..3.2%..3.3%..3.4%..3.5%..3.6%..3.7%..3.8%..3.9%..4.0%..4.1%..4.2%..4.3%..4.4%..4.5%..4.6%..4.7%..4.8%..4.9%..5.0%..5.1%..5.2%..5.3%..5.4%..5.5%..5.6%..5.7%..5.8%..5.9%..6.0%..6.1%..6.2%..6.3%..6.4%..6.5%..6.6%..6.7%..6.8%..6.9%..7.0%..7.1%..7.2%..7.3%..7.4%..7.5%..7.6%..7.7%..7.8%..7.9%..8.0%..8.1%..8.2%..8.3%..8.4%..8.5%..8.6%..8.7%..8.8%..8.9%..9.0%..9.1%..9.2%..9.3%..9.4%..9.5%..9.6%..9.7%..9.8%..9.9%..10.0%..10.1%..10.2%..10.3%..10.4%..10.5%..10.6%..10.7%..10.8%..10.9%..11.0%..11.1%..11.2%..11.3%..11.4%..11.5%..11.6%..11.7%..11.8%..11.9%..12.0%..12.1%..12.2%..12.3%..12.4%..12.5%..12.6%..12.7%..12.8%..12.9%..13.0%..13.1%..13.2%..13.3%..13.4%..13.5%..13.6%..13.7%..13.8%..13.9%..14.0%..14.1%..14.2%..14.3%..14.4%..14.5%..14.6%..14.7%..14.8%..14.9%..15.0%..15.1%..15.2%..15.3%..15.4%..15.5%..15.6%..15.7%..

,index,id,text,source,user_id,truncated,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,retweeted_status_id,geo,place,contributors,retweet_count,reply_count,favorite_count,favorited,retweeted,possibly_sensitive,num_hashtags,num_urls,num_mentions,created_at,timestamp,crawled_at,updated,languages,confidence
0,0,573776239565012992,"If someone asks me to draw my heart,it would b...","<a href=""http://twitter.com/download/android"" ...",1.966433e+09,NaN,0.000000e+00,0.0,NaN,0.000000e+00,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,Fri Mar 06 09:24:50 +0000 2015,2015-03-06 10:24:50,2015-05-01 13:39:11,2015-05-01 13:39:11,ENGLISH,18
1,1,582444087346769920,"@Ironfistmonk Lol, shhh, be careful my dear!","<a href=""http://twitter.com/download/android"" ...",2.668382e+07,NaN,5.824426e+17,596289878.0,Ironfistmonk,0.000000e+00,NaN,NaN,NaN,0.0,0.0,1.0,NaN,NaN,NaN,0.0,0.0,1.0,Mon Mar 30 07:27:46 +0000 2015,2015-03-30 09:27:46,2015-05-01 16:09:45,2015-05-01 16:09:45,ENGLISH,18
2,2,582980402178068481,Let these thick shawty's know “@iAmMeganNichol...,"<a href=""http://twitter.com/download/iphone"" r...",5.387830e+08,NaN,5.829802e+17,525843194.0,iAmMeganNichole,0.000000e+00,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,1.0,Tue Mar 31 18:58:53 +0000 2015,2015-03-31 20:58:53,2015-05-01 13:40:32,2015-05-01 13:40:32,ENGLISH,18
3,3,147662078927310849,kalagot ra! kinsa tong dili nimo ganahan! mao ...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",1.660461e+08,NaN,0.000000e+00,0.0,NaN,0.000000e+00,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,Fri Dec 16 12:59:17 +0000 2011,2011-12-16 13:59:17,2015-05-01 13:24:22,2015-05-01 13:24:22,SPANISH,59
4,4,567135483998646273,"Confirmed na jd, @shinichi_lou.","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",2.300612e+09,NaN,0.000000e+00,0.0,NaN,0.000000e+00,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,1.0,Mon Feb 16 01:36:50 +0000 2015,2015-02-16 02:36:50,2015-05-01 15:59:00,2015-05-01 15:59:00,ENGLISH,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,593645625503129600,RT @ArielleAguilar: @davvvid__ no mames ya vet...,"<a href=""http://twitter.com/download/iphone"" r...",2.286244e+09,NaN,0.000000e+00,0.0,NaN,5.936456e+17,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,2.0,Thu Apr 30 05:18:41 +0000 2015,2015-04-30 07:18:41,2015-05-01 15:52:15,2015-05-01 15:52:15,ENGLISH,18
19996,19996,178531670117462016,@bigtimerush I WANT THEM&lt;3 &lt;3 &lt;3 DEET...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",1.851659e+08,NaN,1.785311e+17,79536438.0,bigtimerush,0.000000e+00,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,1.0,Sat Mar 10 17:24:01 +0000 2012,2012-03-10 18:24:01,2015-05-01 13:45:36,2015-05-01 13:45:36,ENGLISH,18
19997,19997,437559172041674753,@jantlers love you too huhu,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",2.525895e+07,NaN,4.375588e+17,406834561.0,jansucks,0.000000e+00,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,Sun Feb 23 12:06:50 +0000 2014,2014-02-23 13:06:50,2015-05-01 16:02:37,2015-05-01 16:02:37,ENGLISH,18
19998,19998,564517203664326656,Stuuuuuuuff... http://t.co/P06t6AeVum,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",1.844138e+07,NaN,0.000000e+00,0.0,NaN,0.000000e+00,NaN,NaN,NaN,0.0,0.0,3.0,NaN,NaN,NaN,0.0,0.0,0.0,Sun Feb 08 20:12:44 +0000 2015,2015-02-08 21:12:44,2015-05-01 13:53:54,2015-05-01 13:53:54,ENGLISH,18


In [66]:
language_list = {}
for items in genuine_tweets['languages']:
    if language_list.get(str(items)) is None:
        language_list[str(items)] = 1
    else:
        language_list[str(items)] += 1
language_list = pd.Series(language_list)
language_list

ENGLISH     16737
SPANISH      1059
None          792
FRENCH        512
GERMAN        628
JAPANESE      158
ARABIC         67
CHINESE        47
dtype: int64

In [67]:
spambot_tweets = pd.DataFrame()
count = 0
percentage = 0
language_index = {'ENGLISH': 18, 'FRENCH': 22, 'GERMAN': 25, 'SPANISH': 59, 'JAPANESE': 35, 'CHINESE': 5, 'ARABIC': 3, 'None': 7}
for rows in spambots_sheets.sample(n=20000).itertuples():
    count += 1
    last_percentage = percentage
    percentage = floor(count / 20000 * 1000) / 10
    if not (percentage == last_percentage): print("{:.1f}%..".format(percentage), end="")
    row_obj = pd.Series(rows._asdict())
    text_prediction = detector.detect_language_of(msg_clean(str(row_obj['text']), dirty=True))
    row_obj['languages'] = str(text_prediction.name) if text_prediction is not None else 'None'
    row_obj['confidence'] = str(language_index[text_prediction.name]) if text_prediction is not None else 'None'
    if len(spambot_tweets) == 0:
        spambot_tweets = pd.DataFrame([row_obj])
        continue
    spambot_tweets.loc[len(spambot_tweets)] = row_obj
spambot_tweets = spambot_tweets.reset_index().drop('index', axis=1)

0.1%..0.2%..0.3%..0.4%..0.5%..0.6%..0.7%..0.8%..0.9%..1.0%..1.1%..1.2%..1.3%..1.4%..1.5%..1.6%..1.7%..1.8%..1.9%..2.0%..2.1%..2.2%..2.3%..2.4%..2.5%..2.6%..2.7%..2.8%..2.9%..3.0%..3.1%..3.2%..3.3%..3.4%..3.5%..3.6%..3.7%..3.8%..3.9%..4.0%..4.1%..4.2%..4.3%..4.4%..4.5%..4.6%..4.7%..4.8%..4.9%..5.0%..5.1%..5.2%..5.3%..5.4%..5.5%..5.6%..5.7%..5.8%..5.9%..6.0%..6.1%..6.2%..6.3%..6.4%..6.5%..6.6%..6.7%..6.8%..6.9%..7.0%..7.1%..7.2%..7.3%..7.4%..7.5%..7.6%..7.7%..7.8%..7.9%..8.0%..8.1%..8.2%..8.3%..8.4%..8.5%..8.6%..8.7%..8.8%..8.9%..9.0%..9.1%..9.2%..9.3%..9.4%..9.5%..9.6%..9.7%..9.8%..9.9%..10.0%..10.1%..10.2%..10.3%..10.4%..10.5%..10.6%..10.7%..10.8%..10.9%..11.0%..11.1%..11.2%..11.3%..11.4%..11.5%..11.6%..11.7%..11.8%..11.9%..12.0%..12.1%..12.2%..12.3%..12.4%..12.5%..12.6%..12.7%..12.8%..12.9%..13.0%..13.1%..13.2%..13.3%..13.4%..13.5%..13.6%..13.7%..13.8%..13.9%..14.0%..14.1%..14.2%..14.3%..14.4%..14.5%..14.6%..14.7%..14.8%..14.9%..15.0%..15.1%..15.2%..15.3%..15.4%..15.5%..15.6%..15.7%..

In [68]:
spam_language_list = {}
for items in spambot_tweets['languages']:
    if spam_language_list.get(str(items)) is None:
        spam_language_list[str(items)] = 1
    else:
        spam_language_list[str(items)] += 1
spam_language_list = pd.Series(spam_language_list)
spam_language_list

GERMAN        583
ENGLISH     11952
SPANISH      5203
FRENCH       1922
None          314
ARABIC          9
JAPANESE       17
dtype: int64

In [69]:
spambot_tweets.sample(n=15)

,Index,id,text,source,user_id,truncated,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,retweeted_status_id,geo,place,contributors,retweet_count,reply_count,favorite_count,favorited,retweeted,possibly_sensitive,num_hashtags,num_urls,num_mentions,created_at,timestamp,crawled_at,updated,languages,confidence
2998,129306,460110010270232576,I need to find someone over here that will jus...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",2352865495,NaN,0,0,NaN,0,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,0,0,0,Sat Apr 26 17:35:49 +0000 2014,2014-04-26 19:35:49,2014-11-13 02:20:51,2014-11-13 02:20:51,ENGLISH,18
18342,1025912,527790110205820928,1. Chi siamo? 2. Da dove veniamo? 3. Dove stia...,"<a href=""http://www.ErwinBeaver.com"" rel=""nofo...",539178618,NaN,0,0,NaN,0,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,0,0,0,Thu Oct 30 11:52:22 +0000 2014,2014-10-30 12:52:22,2014-11-13 11:53:09,2014-11-13 11:53:09,SPANISH,59
16350,904794,513702526437588993,"Style is knowing who you are, what you want to...","<a href=""http://tweetadder.com"" rel=""nofollow""...",531144071,NaN,0,0,NaN,0,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,0,0,0,Sun Sep 21 14:53:21 +0000 2014,2014-09-21 16:53:21,2014-11-13 04:02:30,2014-11-13 04:02:30,ENGLISH,18
18803,151480,483655250591506432,"@aravindkrish30 ""Take a deep breath, count to ...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",2353721617,NaN,483655090192920576,1286028085,aravindkrish30,0,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,0,0,1,Mon Jun 30 16:56:12 +0000 2014,2014-06-30 18:56:12,2014-11-13 02:33:43,2014-11-13 02:33:43,ENGLISH,18
7646,1557551,478290934804008960,Le cose più belle devi lottare per averle. E s...,"<a href=""http://www.micaelabarbiera.com"" rel=""...",2275243363,NaN,0,0,NaN,0,NaN,NaN,NaN,1,0,12,NaN,NaN,NaN,0,0,0,Sun Jun 15 21:40:19 +0000 2014,2014-06-15 23:40:19,2014-11-13 23:18:01,2014-11-13 23:18:01,SPANISH,59
10937,505136,528515806460915712,"""Il fatto è che tu sei stata ferita. E una per...","<a href=""http://www.aminamilanesi.it"" rel=""nof...",466393676,NaN,0,0,NaN,0,NaN,NaN,NaN,0,0,3,NaN,NaN,NaN,0,0,0,Sat Nov 01 11:56:02 +0000 2014,2014-11-01 12:56:02,2014-11-13 07:49:31,2014-11-13 07:49:31,GERMAN,25
1403,1181132,521486500262055936,On of life's little ironies is the fact that w...,"<a href=""http://tweetadder.com"" rel=""nofollow""...",531246711,NaN,0,0,NaN,0,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,0,0,0,Mon Oct 13 02:24:05 +0000 2014,2014-10-13 04:24:05,2014-11-13 05:31:43,2014-11-13 05:31:43,ENGLISH,18
15660,425323,500616595040141312,I pipistrelli restano appesi sottosopra anche ...,"<a href=""http://www.faigamorgana.com"" rel=""nof...",466320191,NaN,0,0,NaN,0,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,0,0,0,Sat Aug 16 12:14:31 +0000 2014,2014-08-16 14:14:31,2014-11-13 07:24:30,2014-11-13 07:24:30,FRENCH,22
4812,765004,520393927346913280,Read the History of Andy Kim http://t.co/0Ib3...,"<a href=""http://tweetadder.com"" rel=""nofollow""...",522351391,NaN,0,0,NaN,0,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,0,1,0,Fri Oct 10 02:02:35 +0000 2014,2014-10-10 04:02:35,2014-11-13 03:17:28,2014-11-13 03:17:28,ENGLISH,18
2595,300917,530495624358428673,Make the customer the hero of your story. Ann...,"<a href=""http://tweetadder.com"" rel=""nofollow""...",241300883,NaN,0,0,NaN,0,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,0,0,0,Thu Nov 06 23:03:07 +0000 2014,2014-11-07 00:03:07,2014-11-13 00:06:24,2014-11-13 00:06:24,ENGLISH,18


In [70]:
genuine_tweets_copy = genuine_tweets.copy(deep=True)
genuine_tweets_copy.insert(2, 'clean_text', '')
genuine_tweets_copy.insert(3, 'entropy', 0)
genuine_tweets_copy.insert(4, 'word_count', 0)
genuine_tweets_copy.insert(5, 'word_isolated', 0)
genuine_tweets_copy.insert(6, 'average_vec', 0)

for i in range(300):
    genuine_tweets_copy.insert(7 + i, 'wordvec_' + str(i), 0)

count = 0
percentage = 0
for index, rows in genuine_tweets_copy.iterrows():
    count += 1
    last_percentage = percentage
    percentage = floor(count / len(genuine_tweets_copy) * 1000) / 10
    if not (percentage == last_percentage): print("{:.1f}%..".format(percentage), end="")
    rows['clean_text'] = msg_clean(str(rows['text']))
    rows['entropy'] = get_entropy(str(rows['clean_text']))
    rows['word_count'] = len([x for x in jieba.cut(rows['clean_text'], cut_all=False) if not x == ' '])
    word_list = jieba.cut(rows['clean_text'], cut_all=False)
    word_list_trimmed = [x.strip() for x in word_list if x != " "]
    rows['word_isolated'] = ' '.join(word_list_trimmed)
    
    average_vec = np.zeros(300)

    if len(word_list_trimmed) != 0:
        for item in word_list_trimmed:
            try:
                average_vec = average_vec + wv[item]
            except KeyError:
                pass
    
        average_vec = average_vec / len(word_list_trimmed)

    for i in range(300):
        rows["wordvec_" + str(i)] = average_vec[i]
    
    rows["average_vec"] = str(average_vec)
    genuine_tweets_copy.loc[index] = rows
    
genuine_tweets_copy = genuine_tweets_copy.drop('index', axis=1)

C:\UserData\AppData\Local\Temp\ipykernel_3836\244316381.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  genuine_tweets_copy.insert(7 + i, 'wordvec_' + str(i), 0)
C:\UserData\AppData\Local\Temp\ipykernel_3836\244316381.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  genuine_tweets_copy.insert(7 + i, 'wordvec_' + str(i), 0)
C:\UserData\AppData\Local\Temp\ipykernel_3836\244316381.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perform

Loading model from cache C:\UserData\AppData\Local\Temp\jieba.ud72027f4e4a31b625537ac2a72501787.cache
Loading model cost 1.003 seconds.
Prefix dict has been built successfully.


0.1%..0.2%..0.3%..0.4%..0.5%..0.6%..0.7%..0.8%..0.9%..1.0%..1.1%..1.2%..1.3%..1.4%..1.5%..1.6%..1.7%..1.8%..1.9%..2.0%..2.1%..2.2%..2.3%..2.4%..2.5%..2.6%..2.7%..2.8%..2.9%..3.0%..3.1%..3.2%..3.3%..3.4%..3.5%..3.6%..3.7%..3.8%..3.9%..4.0%..4.1%..4.2%..4.3%..4.4%..4.5%..4.6%..4.7%..4.8%..4.9%..5.0%..5.1%..5.2%..5.3%..5.4%..5.5%..5.6%..5.7%..5.8%..5.9%..6.0%..6.1%..6.2%..6.3%..6.4%..6.5%..6.6%..6.7%..6.8%..6.9%..7.0%..7.1%..7.2%..7.3%..7.4%..7.5%..7.6%..7.7%..7.8%..7.9%..8.0%..8.1%..8.2%..8.3%..8.4%..8.5%..8.6%..8.7%..8.8%..8.9%..9.0%..9.1%..9.2%..9.3%..9.4%..9.5%..9.6%..9.7%..9.8%..9.9%..10.0%..10.1%..10.2%..10.3%..10.4%..10.5%..10.6%..10.7%..10.8%..10.9%..11.0%..11.1%..11.2%..11.3%..11.4%..11.5%..11.6%..11.7%..11.8%..11.9%..12.0%..12.1%..12.2%..12.3%..12.4%..12.5%..12.6%..12.7%..12.8%..12.9%..13.0%..13.1%..13.2%..13.3%..13.4%..13.5%..13.6%..13.7%..13.8%..13.9%..14.0%..14.1%..14.2%..14.3%..14.4%..14.5%..14.6%..14.7%..14.8%..14.9%..15.0%..15.1%..15.2%..15.3%..15.4%..15.5%..15.6%..15.7%..

In [86]:
genuine_tweets_copy.sample(n=20)

,id,clean_text,entropy,word_count,word_isolated,average_vec,wordvec_0,wordvec_1,wordvec_2,wordvec_3,wordvec_4,wordvec_5,wordvec_6,wordvec_7,wordvec_8,wordvec_9,wordvec_10,wordvec_11,wordvec_12,wordvec_13,wordvec_14,wordvec_15,wordvec_16,wordvec_17,wordvec_18,wordvec_19,wordvec_20,wordvec_21,wordvec_22,wordvec_23,wordvec_24,wordvec_25,wordvec_26,wordvec_27,wordvec_28,wordvec_29,wordvec_30,wordvec_31,wordvec_32,wordvec_33,wordvec_34,wordvec_35,wordvec_36,wordvec_37,wordvec_38,wordvec_39,wordvec_40,wordvec_41,wordvec_42,wordvec_43,wordvec_44,wordvec_45,wordvec_46,wordvec_47,wordvec_48,wordvec_49,wordvec_50,wordvec_51,wordvec_52,wordvec_53,wordvec_54,wordvec_55,wordvec_56,wordvec_57,wordvec_58,wordvec_59,wordvec_60,wordvec_61,wordvec_62,wordvec_63,wordvec_64,wordvec_65,wordvec_66,wordvec_67,wordvec_68,wordvec_69,wordvec_70,wordvec_71,wordvec_72,wordvec_73,wordvec_74,wordvec_75,wordvec_76,wordvec_77,wordvec_78,wordvec_79,wordvec_80,wordvec_81,wordvec_82,wordvec_83,wordvec_84,wordvec_85,wordvec_86,wordvec_87,wordvec_88,wordvec_89,wordvec_90,wordvec_91,wordvec_92,wordvec_93,wordvec_94,wordvec_95,wordvec_96,wordvec_97,wordvec_98,wordvec_99,wordvec_100,wordvec_101,wordvec_102,wordvec_103,wordvec_104,wordvec_105,wordvec_106,wordvec_107,wordvec_108,wordvec_109,wordvec_110,wordvec_111,wordvec_112,wordvec_113,wordvec_114,wordvec_115,wordvec_116,wordvec_117,wordvec_118,wordvec_119,wordvec_120,wordvec_121,wordvec_122,wordvec_123,wordvec_124,wordvec_125,wordvec_126,wordvec_127,wordvec_128,wordvec_129,wordvec_130,wordvec_131,wordvec_132,wordvec_133,wordvec_134,wordvec_135,wordvec_136,wordvec_137,wordvec_138,wordvec_139,wordvec_140,wordvec_141,wordvec_142,wordvec_143,wordvec_144,wordvec_145,wordvec_146,wordvec_147,wordvec_148,wordvec_149,wordvec_150,wordvec_151,wordvec_152,wordvec_153,wordvec_154,wordvec_155,wordvec_156,wordvec_157,wordvec_158,wordvec_159,wordvec_160,wordvec_161,wordvec_162,wordvec_163,wordvec_164,wordvec_165,wordvec_166,wordvec_167,wordvec_168,wordvec_169,wordvec_170,wordvec_171,wordvec_172,wordvec_173,wordvec_174,wordvec_175,wordvec_176,wordvec_177,wordvec_178,wordvec_179,wordvec_180,wordvec_181,wordvec_182,wordvec_183,wordvec_184,wordvec_185,wordvec_186,wordvec_187,wordvec_188,wordvec_189,wordvec_190,wordvec_191,wordvec_192,wordvec_193,wordvec_194,wordvec_195,wordvec_196,wordvec_197,wordvec_198,wordvec_199,wordvec_200,wordvec_201,wordvec_202,wordvec_203,wordvec_204,wordvec_205,wordvec_206,wordvec_207,wordvec_208,wordvec_209,wordvec_210,wordvec_211,wordvec_212,wordvec_213,wordvec_214,wordvec_215,wordvec_216,wordvec_217,wordvec_218,wordvec_219,wordvec_220,wordvec_221,wordvec_222,wordvec_223,wordvec_224,wordvec_225,wordvec_226,wordvec_227,wordvec_228,wordvec_229,wordvec_230,wordvec_231,wordvec_232,wordvec_233,wordvec_234,wordvec_235,wordvec_236,wordvec_237,wordvec_238,wordvec_239,wordvec_240,wordvec_241,wordvec_242,wordvec_243,wordvec_244,wordvec_245,wordvec_246,wordvec_247,wordvec_248,wordvec_249,wordvec_250,wordvec_251,wordvec_252,wordvec_253,wordvec_254,wordvec_255,wordvec_256,wordvec_257,wordvec_258,wordvec_259,wordvec_260,wordvec_261,wordvec_262,wordvec_263,wordvec_264,wordvec_265,wordvec_266,wordvec_267,wordvec_268,wordvec_269,wordvec_270,wordvec_271,wordvec_272,wordvec_273,wordvec_274,wordvec_275,wordvec_276,wordvec_277,wordvec_278,wordvec_279,wordvec_280,wordvec_281,wordvec_282,wordvec_283,wordvec_284,wordvec_285,wordvec_286,wordvec_287,wordvec_288,wordvec_289,wordvec_290,wordvec_291,wordvec_292,wordvec_293,wordvec_294,wordvec_295,wordvec_296,wordvec_297,wordvec_298,wordvec_299,text,source,user_id,truncated,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,retweeted_status_id,geo,place,contributors,retweet_count,reply_count,favorite_count,favorited,retweeted,possibly_sensitive,num_hashtags,num_urls,num_mentions,created_at,timestamp,crawled_at,updated,languages,confidence,is_spam
17333,486273281209552896,A Category 5 barely missed my city last year w...,2.890372,18,A Category 5 barely missed my city last 

In [99]:
spambot_tweets_copy = spambot_tweets.copy(deep=True)
spambot_tweets_copy.insert(2, 'clean_text', '')
spambot_tweets_copy.insert(3, 'entropy', '0')
spambot_tweets_copy.insert(4, 'word_count', 0)
spambot_tweets_copy.insert(5, 'word_isolated', 0)
spambot_tweets_copy.insert(6, 'average_vec', 0)

for i in range(300):
    spambot_tweets_copy.insert(7 + i, 'wordvec_' + str(i), 0)

count = 0
percentage = 0
for index, rows in spambot_tweets_copy.iterrows():
    count += 1
    last_percentage = percentage
    percentage = floor(count / len(genuine_tweets_copy) * 1000) / 10
    if not (percentage == last_percentage): print("{:.1f}%..".format(percentage), end="")
    rows['clean_text'] = msg_clean(str(rows['text']))
    rows['entropy'] = get_entropy(str(rows['clean_text']))
    rows['word_count'] = len([x for x in jieba.cut(rows['clean_text'], cut_all=False) if not x == ' '])
    word_list = jieba.cut(rows['clean_text'], cut_all=False)
    word_list_trimmed = [x.strip() for x in word_list if x != " "]
    rows['word_isolated'] = ' '.join(word_list_trimmed)
    
    average_vec = np.zeros(300)

    if len(word_list_trimmed) != 0:
        for item in word_list_trimmed:
            try:
                average_vec = average_vec + wv[item]
            except KeyError:
                pass
    
        average_vec = average_vec / len(word_list_trimmed)

    for i in range(300):
        rows["wordvec_" + str(i)] = average_vec[i]
    
    rows["average_vec"] = str(average_vec)
    spambot_tweets_copy.loc[index] = rows

C:\UserData\AppData\Local\Temp\ipykernel_3836\3764762691.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  spambot_tweets_copy.insert(7 + i, 'wordvec_' + str(i), 0)
C:\UserData\AppData\Local\Temp\ipykernel_3836\3764762691.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  spambot_tweets_copy.insert(7 + i, 'wordvec_' + str(i), 0)
C:\UserData\AppData\Local\Temp\ipykernel_3836\3764762691.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perf

0.1%..0.2%..0.3%..0.4%..0.5%..0.6%..0.7%..0.8%..0.9%..1.0%..1.1%..1.2%..1.3%..1.4%..1.5%..1.6%..1.7%..1.8%..1.9%..2.0%..2.1%..2.2%..2.3%..2.4%..2.5%..2.6%..2.7%..2.8%..2.9%..3.0%..3.1%..3.2%..3.3%..3.4%..3.5%..3.6%..3.7%..3.8%..3.9%..4.0%..4.1%..4.2%..4.3%..4.4%..4.5%..4.6%..4.7%..4.8%..4.9%..5.0%..5.1%..5.2%..5.3%..5.4%..5.5%..5.6%..5.7%..5.8%..5.9%..6.0%..6.1%..6.2%..6.3%..6.4%..6.5%..6.6%..6.7%..6.8%..6.9%..7.0%..7.1%..7.2%..7.3%..7.4%..7.5%..7.6%..7.7%..7.8%..7.9%..8.0%..8.1%..8.2%..8.3%..8.4%..8.5%..8.6%..8.7%..8.8%..8.9%..9.0%..9.1%..9.2%..9.3%..9.4%..9.5%..9.6%..9.7%..9.8%..9.9%..10.0%..10.1%..10.2%..10.3%..10.4%..10.5%..10.6%..10.7%..10.8%..10.9%..11.0%..11.1%..11.2%..11.3%..11.4%..11.5%..11.6%..11.7%..11.8%..11.9%..12.0%..12.1%..12.2%..12.3%..12.4%..12.5%..12.6%..12.7%..12.8%..12.9%..13.0%..13.1%..13.2%..13.3%..13.4%..13.5%..13.6%..13.7%..13.8%..13.9%..14.0%..14.1%..14.2%..14.3%..14.4%..14.5%..14.6%..14.7%..14.8%..14.9%..15.0%..15.1%..15.2%..15.3%..15.4%..15.5%..15.6%..15.7%..

In [124]:
spambot_tweets_copy['is_spam'] = 1
spambot_tweets_copy.sample(n=20).drop('Index', 1)

C:\UserData\AppData\Local\Temp\ipykernel_3836\2942179785.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  spambot_tweets_copy.sample(n=20).drop('Index', 1)


,id,clean_text,entropy,word_count,word_isolated,average_vec,wordvec_0,wordvec_1,wordvec_2,wordvec_3,wordvec_4,wordvec_5,wordvec_6,wordvec_7,wordvec_8,wordvec_9,wordvec_10,wordvec_11,wordvec_12,wordvec_13,wordvec_14,wordvec_15,wordvec_16,wordvec_17,wordvec_18,wordvec_19,wordvec_20,wordvec_21,wordvec_22,wordvec_23,wordvec_24,wordvec_25,wordvec_26,wordvec_27,wordvec_28,wordvec_29,wordvec_30,wordvec_31,wordvec_32,wordvec_33,wordvec_34,wordvec_35,wordvec_36,wordvec_37,wordvec_38,wordvec_39,wordvec_40,wordvec_41,wordvec_42,wordvec_43,wordvec_44,wordvec_45,wordvec_46,wordvec_47,wordvec_48,wordvec_49,wordvec_50,wordvec_51,wordvec_52,wordvec_53,wordvec_54,wordvec_55,wordvec_56,wordvec_57,wordvec_58,wordvec_59,wordvec_60,wordvec_61,wordvec_62,wordvec_63,wordvec_64,wordvec_65,wordvec_66,wordvec_67,wordvec_68,wordvec_69,wordvec_70,wordvec_71,wordvec_72,wordvec_73,wordvec_74,wordvec_75,wordvec_76,wordvec_77,wordvec_78,wordvec_79,wordvec_80,wordvec_81,wordvec_82,wordvec_83,wordvec_84,wordvec_85,wordvec_86,wordvec_87,wordvec_88,wordvec_89,wordvec_90,wordvec_91,wordvec_92,wordvec_93,wordvec_94,wordvec_95,wordvec_96,wordvec_97,wordvec_98,wordvec_99,wordvec_100,wordvec_101,wordvec_102,wordvec_103,wordvec_104,wordvec_105,wordvec_106,wordvec_107,wordvec_108,wordvec_109,wordvec_110,wordvec_111,wordvec_112,wordvec_113,wordvec_114,wordvec_115,wordvec_116,wordvec_117,wordvec_118,wordvec_119,wordvec_120,wordvec_121,wordvec_122,wordvec_123,wordvec_124,wordvec_125,wordvec_126,wordvec_127,wordvec_128,wordvec_129,wordvec_130,wordvec_131,wordvec_132,wordvec_133,wordvec_134,wordvec_135,wordvec_136,wordvec_137,wordvec_138,wordvec_139,wordvec_140,wordvec_141,wordvec_142,wordvec_143,wordvec_144,wordvec_145,wordvec_146,wordvec_147,wordvec_148,wordvec_149,wordvec_150,wordvec_151,wordvec_152,wordvec_153,wordvec_154,wordvec_155,wordvec_156,wordvec_157,wordvec_158,wordvec_159,wordvec_160,wordvec_161,wordvec_162,wordvec_163,wordvec_164,wordvec_165,wordvec_166,wordvec_167,wordvec_168,wordvec_169,wordvec_170,wordvec_171,wordvec_172,wordvec_173,wordvec_174,wordvec_175,wordvec_176,wordvec_177,wordvec_178,wordvec_179,wordvec_180,wordvec_181,wordvec_182,wordvec_183,wordvec_184,wordvec_185,wordvec_186,wordvec_187,wordvec_188,wordvec_189,wordvec_190,wordvec_191,wordvec_192,wordvec_193,wordvec_194,wordvec_195,wordvec_196,wordvec_197,wordvec_198,wordvec_199,wordvec_200,wordvec_201,wordvec_202,wordvec_203,wordvec_204,wordvec_205,wordvec_206,wordvec_207,wordvec_208,wordvec_209,wordvec_210,wordvec_211,wordvec_212,wordvec_213,wordvec_214,wordvec_215,wordvec_216,wordvec_217,wordvec_218,wordvec_219,wordvec_220,wordvec_221,wordvec_222,wordvec_223,wordvec_224,wordvec_225,wordvec_226,wordvec_227,wordvec_228,wordvec_229,wordvec_230,wordvec_231,wordvec_232,wordvec_233,wordvec_234,wordvec_235,wordvec_236,wordvec_237,wordvec_238,wordvec_239,wordvec_240,wordvec_241,wordvec_242,wordvec_243,wordvec_244,wordvec_245,wordvec_246,wordvec_247,wordvec_248,wordvec_249,wordvec_250,wordvec_251,wordvec_252,wordvec_253,wordvec_254,wordvec_255,wordvec_256,wordvec_257,wordvec_258,wordvec_259,wordvec_260,wordvec_261,wordvec_262,wordvec_263,wordvec_264,wordvec_265,wordvec_266,wordvec_267,wordvec_268,wordvec_269,wordvec_270,wordvec_271,wordvec_272,wordvec_273,wordvec_274,wordvec_275,wordvec_276,wordvec_277,wordvec_278,wordvec_279,wordvec_280,wordvec_281,wordvec_282,wordvec_283,wordvec_284,wordvec_285,wordvec_286,wordvec_287,wordvec_288,wordvec_289,wordvec_290,wordvec_291,wordvec_292,wordvec_293,wordvec_294,wordvec_295,wordvec_296,wordvec_297,wordvec_298,wordvec_299,text,source,user_id,truncated,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,retweeted_status_id,geo,place,contributors,retweet_count,reply_count,favorite_count,favorited,retweeted,possibly_sensitive,num_hashtags,num_urls,num_mentions,created_at,timestamp,crawled_at,updated,languages,confidence,is_spam
4338,514074195480023040,I really love this Han Solo iPhone case may t...,2.639057,14,I really love this Han Solo iPhone case ma

In [131]:
language_index = {'ENGLISH': 0, 'FRENCH': 1, 'GERMAN': 2, 'SPANISH': 3, 'JAPANESE': 4, 'CHINESE': 5, 'ARABIC': 6, 'None': 7}
#genuine_tweets_copy['languages'] = genuine_tweets_copy['languages'].apply((lambda x: language_index[str(x)]))
#spambot_tweets_copy['languages'] = spambot_tweets_copy['languages'].apply((lambda x: language_index[str(x)]))
print(spambot_tweets_copy['languages'])

0        2
1        0
2        0
3        0
4        3
        ..
19995    0
19996    0
19997    0
19998    7
19999    0
Name: languages, Length: 20000, dtype: int64


In [132]:
genuine_tweets_copy['is_spam'] = 0
spambot_tweets_copy['is_spam'] = 1

In [133]:
tweets_copy = pd.concat([genuine_tweets_copy, spambot_tweets_copy]).reset_index().drop('index', axis=1)

In [134]:
tweets_copy.sample(n=1).reset_index

<bound method DataFrame.reset_index of                        id  clean_text   entropy  word_count word_isolated  \
13057  459871531762343937   I DMd you  1.098612           3     I DMd you   

                                             average_vec  wordvec_0  \
13057  [ 9.44010417e-02  2.71606445e-03  6.25000000e-...   0.094401   

       wordvec_1  wordvec_2  wordvec_3  wordvec_4  wordvec_5  wordvec_6  \
13057   0.002716     0.0625   0.166016   0.007324   0.028788  -0.021647   

       wordvec_7  wordvec_8  wordvec_9  wordvec_10  wordvec_11  wordvec_12  \
13057  -0.009847  -0.023519    0.03716   -0.030434   -0.116211   -0.041768   

       wordvec_13  wordvec_14  wordvec_15  wordvec_16  wordvec_17  wordvec_18  \
13057   -0.103841   -0.146484    0.073568    0.058594    0.025065    0.131185   

       wordvec_19  wordvec_20  wordvec_21  wordvec_22  wordvec_23  wordvec_24  \
13057   -0.054036   -0.007161    0.113607    0.173828   -0.059896   -0.028483   

       wordvec_25  wordvec_26

In [135]:
tweets_feature_list = ['entropy', 'word_count', 'retweet_count', 'reply_count', 'favorite_count', 'num_hashtags', 'num_urls', 'num_mentions', 'languages']
tweets_model_npy = tweets_copy[tweets_feature_list].to_numpy()
tweets_vector_npy = tweets_copy[["wordvec_" + str(x) for x in range(300)]].to_numpy()
tweets_result_npy = tweets_copy['is_spam'].to_numpy()

In [136]:
tweets_model_with_vector = np.hstack([tweets_model_npy, tweets_vector_npy])

In [137]:
tweets_model_with_vector

array([[2.4849066497880012, 12, 0.0, ..., -0.054735819498697914,
        0.0014979044596354167, -0.047465006510416664],
       [1.7917594692280547, 6, 0.0, ..., -0.079986572265625,
        -0.060384114583333336, 0.054606119791666664],
       [2.70805020110221, 15, 1.0, ..., -0.04366861979166667,
        -0.046830240885416666, 0.010001627604166667],
       ...,
       [2.0794415416798357, 8, 0.0, ..., -0.024358749389648438,
        0.04962158203125, -0.039947509765625],
       [0, 0, 0.0, ..., 0.0, 0.0, 0.0],
       [2.0258552616175165, 8, 0.0, ..., -0.05309295654296875,
        -0.002105712890625, -0.107666015625]], dtype=object)

In [138]:
X_train, X_test, y_train, y_test = train_test_split(tweets_model_with_vector, tweets_result_npy, random_state=114514)

In [139]:
X_train.shape

(30000, 309)

In [140]:
print(X_test)

[[2.576457673843677 13 0.0 ... -0.019385704627403848 0.015047513521634616
  -0.02416053185096154]
 [1.6094379124341005 5 0.0 ... -0.09326171875 -0.0208984375 -0.0072265625]
 [2.7961672571010316 16 0.0 ... -0.0834808349609375 -0.021053314208984375
  0.02906036376953125]
 ...
 [0.6931471805599453 2 0.0 ... -0.10888671875 0.052001953125 0.0751953125]
 [2.973931613272625 19 25.0 ... -0.07183677271792763
  -0.0016704358552631579 -0.0015861109683388158]
 [3.1685310030055156 23 0.0 ... -0.07032908564028532 0.07907369862432065
  0.014522718346637228]]


In [141]:
treeObj = RandomForestClassifier(n_estimators=60, random_state=14514, max_depth=7)
treeObj.fit(X_train, y_train)

RandomForestClassifier(max_depth=7, n_estimators=60, random_state=14514)

In [142]:
treeObj.score(X_train, y_train)

0.8949666666666667

In [31]:
treeObj.score(X_test, y_test)

0.8778

In [143]:
pre = treeObj.predict(X_test)
print(classification_report(y_test, pre))

              precision    recall  f1-score   support

           0       0.85      0.94      0.89      4986
           1       0.93      0.83      0.88      5014

    accuracy                           0.88     10000
   macro avg       0.89      0.88      0.88     10000
weighted avg       0.89      0.88      0.88     10000



使用SVC再次進行擬合

```
{'original_text': 'I died, I thought #it #was #Metroid and I almost <f=http://twitter.isayadmin.com/ @KushyD @KushyD @KushyD', 'cleaned_text': 'I died I thought    and I almost f   ', 'word_count': 11, 'num_urls': 1, 'num_hashtags': 3, 'num_mentions': 3, 'num_retweets': 15, 'num_replys': 5, 'num_favorite': 3, 'lang_code': 0, 'entropy': 1.8397064813033515}
```


In [144]:
SVCObj = SVC(kernel="rbf", C=1, gamma="auto")

In [145]:
SVCObj.fit(X_train, y_train)

SVC(C=1, gamma='auto')

In [146]:
SVCpre = SVCObj.predict(X_test)
print(classification_report(y_test, SVCpre))

              precision    recall  f1-score   support

           0       0.88      0.82      0.85      4986
           1       0.83      0.88      0.86      5014

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000



In [147]:
from sklearn.neighbors import  KNeighborsClassifier

In [149]:
knn = KNeighborsClassifier(n_neighbors=12)
knn.fit(X_train,y_train)
y_pre = knn.predict(X_test)
print(classification_report(y_test,y_pre))

D:\conda\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


              precision    recall  f1-score   support

           0       0.87      0.89      0.88      4986
           1       0.89      0.87      0.88      5014

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000

